In [3]:
import h2o
h2o.init()
h2o.demo("glm")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
  Starting server from /home/biswajitrana/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp8b77hg1i
  JVM stdout: /tmp/tmp8b77hg1i/h2o_biswajitrana_started_from_python.out
  JVM stderr: /tmp/tmp8b77hg1i/h2o_biswajitrana_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_biswajitrana_1vjoo7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.885 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


-------------------------------------------------------------------------------
Demo of H2O's Generalized Linear Estimator.

This demo uploads a dataset to h2o, parses it, and shows a description.
Then it divides the dataset into training and test sets, builds a GLM
from the training set, and makes predictions for the test set.
Finally, default performance metrics are displayed.
-------------------------------------------------------------------------------

>>> # Connect to H2O
>>> h2o.init()

(press any key)

error: (25, 'Inappropriate ioctl for device')

In [4]:
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
  Starting server from /home/biswajitrana/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpe5sx_elv
  JVM stdout: /tmp/tmpe5sx_elv/h2o_biswajitrana_started_from_python.out
  JVM stderr: /tmp/tmpe5sx_elv/h2o_biswajitrana_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_biswajitrana_32gaqm
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.885 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [5]:
loans = h2o.import_file("https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-at-Scale-with-H2O/main/chapt2/loans-lite.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
loans.describe()

Rows:5000
Cols:16

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,credit_length
type,int,enum,real,int,enum,real,enum,int,enum,enum,real,int,real,int,int,int
mins,500.0,,5.42,0.0,,3300.0,,1180656000000.0,,,0.0,0.0,0.0,1.0,0.0,0.0
mean,10757.4,,12.042216,4.740286298568507,,68984.26325465094,,1284307868160.0,,,13.212278,0.14031224979983986,49.15658321650311,21.830064051240992,0.155,13.413130504403522
maxs,35000.0,,23.91,10.0,,3900000.0,,1322697600000.0,,,29.93,6.0,106.5,73.0,1.0,52.0
sigma,7173.197679068143,,3.612830457982719,3.580079530043472,,78076.24567859183,,32089484313.69494,,,6.71513728560101,0.4702652144800979,28.367928141780574,11.634565225391304,0.361940879481785,6.844231383509271
zeros,0,,0,659,,0,,0,,,26,4470,124,0,4225,1
missing,0,0,0,110,0,1,0,0,0,0,0,4,7,4,0,4
0,5000.0,36 months,11.97,1.0,OWN,51320.0,not verified,1207008000000.0,debt_consolidation,CA,22.73,0.0,70.3,11.0,0.0,10.0
1,18000.0,60 months,13.49,10.0,RENT,72000.0,verified,1317427200000.0,debt_consolidation,PA,19.1,0.0,74.8,13.0,0.0,16.0
2,11975.0,60 months,15.21,3.0,MORTGAGE,64584.0,verified,1285891200000.0,debt_consolidation,MD,19.81,0.0,61.1,17.0,0.0,8.0


In [7]:
train, validation = loans.split_frame(ratios=[0.75])
label = "bad_loan"
predictors = loans.col_names
predictors.remove(label)

In [ ]:
from h2o.estimators import H2OXGBoostEstimator
param = {"ntrees" : 25, "nfolds" : 10}
xgboost_model = H2OXGBoostEstimator(**param)
xgboost_model.train(x = predictors,
                    y = label,
                    training_frame = train,
                    validation_frame = validation)